Masks for Countries

In [ ]:
import geopandas as gpd
import xarray as xr
import numpy as np
import os
from shapely.geometry import Point, box

# Grid parameters (same as your dataset)
lat_res = 0.75
lon_res = 0.75
lat_min, lat_max = 34, 66
lon_min, lon_max = -12, 36

# Create lat/lon centers for grid cells
lat_bins = np.arange(lat_min, lat_max + lat_res, lat_res)
lon_bins = np.arange(lon_min, lon_max + lon_res, lon_res)
lat_centers = lat_bins[:-1]
lon_centers = lon_bins[:-1]

# Create 2D meshgrid of pixel centers (longitude, latitude)
lon_grid, lat_grid = np.meshgrid(lon_centers, lat_centers)

# Flatten to 1D arrays for point-in-polygon test
points = np.vstack([lon_grid.ravel(), lat_grid.ravel()]).T

# Input: list of shapefile paths for each region (country)
shapefile_paths = {
    "Portugal": r"E:\IPMA\Countries\PRT\gadm41_PRT_0.shp",
    "Spain": r"E:\IPMA\Countries\ESP\gadm41_ESP_0.shp",
    "Italy": r"E:\IPMA\Countries\ITA\gadm41_ITA_0.shp",
    "Greece": r"E:\IPMA\Countries\GRC\gadm41_GRC_0.shp"
}

# Load all shapefiles into GeoDataFrames
gdfs = {name: gpd.read_file(shp) for name, shp in shapefile_paths.items()}

# Ensure all shapefiles are in WGS84 (EPSG:4326)
for name, gdf in gdfs.items():
    if gdf.crs != "EPSG:4326":
        gdfs[name] = gdf.to_crs("EPSG:4326")

# Create mask arrays for each region, initialize with False
masks = {name: np.zeros(lat_grid.shape, dtype=bool) for name in gdfs}

# Keep track of pixels already assigned (to avoid overlap)
assigned_pixels = np.zeros(lat_grid.shape, dtype=bool)

from shapely.geometry import box

print("Creating pixel polygons...")

# Create pixel polygons (boxes) for each pixel center
half_lat = lat_res / 2
half_lon = lon_res / 2
pixel_polygons = [box(lon - half_lon, lat - half_lat, lon + half_lon, lat + half_lat)
                  for lon, lat in points]

print("Creating masks...")

# 1. Process Portugal and Spain first with exclusive pixel assignment
assigned_pixels = np.zeros(lat_grid.size, dtype=bool)
for name in ["Portugal", "Spain"]:
    gdf = gdfs[name]
    print(f"Processing {name} with exclusive assignment...")

    mask = np.zeros(lat_grid.size, dtype=bool)
    spatial_index = gdf.sindex

    for i, pix_poly in enumerate(pixel_polygons):
        if assigned_pixels[i]:
            continue  # skip pixels already assigned to Portugal or Spain

        possible_matches_index = list(spatial_index.intersection(pix_poly.bounds))
        possible_matches = gdf.iloc[possible_matches_index]

        # Use intersects instead of contains
        if possible_matches.geometry.apply(lambda poly: poly.intersects(pix_poly)).any():
            mask[i] = True

    assigned_pixels = assigned_pixels | mask  # both 1D boolean arrays
    masks[name] = mask.reshape(lat_grid.shape)


# 2. Process Italy and Greece without checking assigned_pixels (allow overlap)
for name in ["Italy", "Greece"]:
    gdf = gdfs[name]
    print(f"Processing {name} without exclusive assignment...")

    mask = np.zeros(lat_grid.size, dtype=bool)
    spatial_index = gdf.sindex

    for i, pix_poly in enumerate(pixel_polygons):
        possible_matches_index = list(spatial_index.intersection(pix_poly.bounds))
        possible_matches = gdf.iloc[possible_matches_index]

        if possible_matches.geometry.apply(lambda poly: poly.intersects(pix_poly)).any():
            mask[i] = True

    mask = mask.reshape(lat_grid.shape)
    masks[name] = mask


# Now save each mask as a NetCDF file matching your grid

# Define output folder for masks
output_folder = r"E:\IPMA\Countries"
os.makedirs(output_folder, exist_ok=True)

# Save masks as NetCDF
for name, mask in masks.items():
    ds_mask = xr.Dataset(
        {
            "mask": (["latitude", "longitude"], mask.astype(int))
        },
        coords={
            "latitude": lat_centers,
            "longitude": lon_centers
        },
        attrs={
            "description": f"Mask for {name}, 1=inside, 0=outside",
            "grid_resolution": f"{lat_res} x {lon_res} degrees",
            "study_area_latitude_range": f"{lat_min} to {lat_max}",
            "study_area_longitude_range": f"{lon_min} to {lon_max}"
        }
    )
    output_path = os.path.join(output_folder, f"{name}_mask.nc")
    print(f"Saving mask for {name} to {output_path}")
    ds_mask.to_netcdf(output_path)


print("Done creating masks.")


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os

# --- Set your study area extent ---
lat_min, lat_max = 34, 66
lon_min, lon_max = -12, 36

# --- Paths to mask NetCDF files ---
# Update these paths if needed
mask_dir = r"E:\IPMA\Countries"  # folder where your mask files are saved

mask_files = {
    "Portugal": os.path.join(mask_dir, "Portugal_mask.nc"),
    "Spain": os.path.join(mask_dir, "Spain_mask.nc"),
    "Italy": os.path.join(mask_dir, "Italy_mask.nc"),
    "Greece": os.path.join(mask_dir, "Greece_mask.nc"),
}

# --- Load masks into dict ---
masks = {country: xr.open_dataset(path) for country, path in mask_files.items()}

# --- Plot ---
fig = plt.figure(figsize=(12, 10))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot each mask with transparency and different color
colors = {
    "Portugal": "forestgreen",
    "Spain": "orange",
    "Italy": "royalblue",
    "Greece": "crimson",
}

for country, ds in masks.items():
    mask = ds['mask']  # variable name depends on your dataset, adjust if different
    # Plot mask where value == 1
    mask_plot = mask.where(mask == 1)
    mask_plot.plot.pcolormesh(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap=plt.cm.get_cmap('tab10'),
        add_colorbar=False,
        alpha=0.5,
        label=country,
        rasterized=True,
    )

# Add coastlines and borders
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Set map extent to your study area
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add gridlines and labels
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.7, linestyle='--')
gl.top_labels = False
gl.right_labels = False

# Legend manually (since pcolormesh doesn’t create legend automatically)
import matplotlib.patches as mpatches
patches = [mpatches.Patch(color=colors[c], label=c) for c in masks.keys()]
plt.legend(handles=patches, loc='upper left')

plt.title("Country Masks Overlay on Study Area")
plt.show()
